[View in Colaboratory](https://colab.research.google.com/github/jsedoc/Tree-Learning-MASC/blob/master/spinntree2seq.ipynb)

First we need to install the necessary libraries

In [0]:
import sys
print(sys.version_info)

!pip install http://download.pytorch.org/whl/cu90/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip install tqdm nltk
import torch

print(torch.cuda.device_count())
use_cuda = torch.cuda.is_available()


In [0]:
import os, sys

Now we are going to mount the google drive folder to load / save checkpoints. 

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials



Next we clone the python files

In [0]:
!git clone https://github.com/jsedoc/Tree-Learning-MASC tl

Now we add the system paths

In [0]:
print("Adding the path to our code in:", os.path.abspath(os.path.join('tl/code')))
sys.path.append(os.path.abspath(os.path.join('tl/code')))
from tl.code.tree2seq import *
from tl.code.tree2seq.spinn import *

Load the data next

In [0]:
enc_filename = os.path.abspath(os.path.join('tl/code/data/train.orig'))
dec_filename = os.path.abspath(os.path.join('tl/code/data/train.orig'))
                               
dataset = Dataset()
dataset.create_vocab(enc_filename)
one_hot_dict = dataset.create_one_hot(128)
trees = dataset.read_trees(enc_filename)
seqs = dataset.read_seqs(dec_filename)
ptr_trees = [dataset.make_tree_seq(tree) for tree in tqdm(trees)]


In [0]:
values = [dataset.token_dict[_] for _ in seqs[1].strip().replace('(','').replace(')','').split()]
buf = torch.stack([torch.stack([one_hot_dict[_].view(-1) for _ in seqs[1].strip().replace('(','').replace(')','').split()])])
transitions = [REDUCE if _ == ')' else SHIFT for _ in seqs[1].strip().replace('(','').split()]

print(buf)
print(transitions)
print(len(buf))

Now we can finally initialize the SPINN

In [0]:
config = SPINNConfig()
config.d_hidden = 128
encoder = SPINN(config)
#print(encoder.forward(ptr_trees[1][0], Var(ptr_trees[1][1])))
print(config.d_hidden)

Run the forward step on the SPINN module

In [0]:
encoder.forward(Variable(buf), Variable(torch.LongTensor(transitions).view(1,-1)))